## t検定
母集団の統計量が不明で、t値を計算して仮説の棄却を検討する流れ

In [1]:
# 必要なライブラリのimport
import numpy as np
import pandas as pd
from scipy import stats

import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_theme()

.squeeze("columns")は、hypothesis-testing-1.csv が1列しかないCSVファイルのため、自動でdata1をSeries型にするためのキーワード引数です。Series型にしない場合、次の mean() の実行で警告が表示されます

In [2]:
# 1変量データのCSVファイルの読み込み
data1 = pd.read_csv('./hypothesis-testing-1.csv').squeeze("columns")

In [3]:
# 標本平均の計算
mu = np.mean(data1)
mu

np.float64(51.82018326803313)

In [4]:
# 自由度の計算
df = len(data1) - 1
df

29

In [5]:
    # 標準誤差の計算
sigma = np.std(data1, ddof=1)
se = sigma/np.sqrt(len(data1))
se

np.float64(0.5621633454110073)

両側検定の実施
標本平均と標準誤差を計算できました。では、この情報を元に「池の魚平均体長は50cmと等しいか」について調べてみましょう。今回は両側検定を採用し、有意水準は5％とします。帰無仮説・対立仮説は以下の通りです。

帰無仮説（H0）: 池の中の魚の平均体長は50cmである
対立仮説（H1）: 池の中の魚の平均体長は50cmとは異なる


In [ ]:
# t値の計算
t_value = (mu-50)/se #muは標本平均、seは標準誤差
t_value

np.float64(3.2378191906168468)

p値の計算
p値の計算には stats.t.cdf() を使います。最初の位置引数にt値、キーワード引数dfに自由度を指定します。

In [7]:
# p値を計算（両側検定：2倍して両側の確率を求める）
# t値の絶対値を取ってからcdfを計算し、上側確率を求めて2倍する
alpha = stats.t.cdf(np.abs(t_value), df=df)
p_value = (1 - alpha) * 2
p_value

np.float64(0.0030122666127776476)

上記のコードについて補足します。

（1）stats.t.cdf(x, df) の計算結果は、「自由度dfのt分布において、変数がx以下の値を取る確率（下側確率）」を表しています。観測されたt値よりも極端な値が得られる確率（p値）を計算するには、このcdfの値や生存関数 sf (1-cdf) を用います。両側検定では、t値の絶対値 np.abs(t_value) を用い、その値以上になる片側の確率を計算し、それを2倍します。

（2）t値は標本データによって正の値にも負の値にもなりえます。p値を計算する際は、帰無仮説からどれだけ離れているか（ズレの大きさ）が問題となるため、多くの場合t値の絶対値 np.abs(t_value) を用いて計算します。

（3）例えば stats.t.sf(np.abs(t_value), df=df) は、t分布において np.abs(t_value) 以上の値を取る確率（片側の上側確率）を示します。両側検定では、この確率を2倍してp値を求めます。

今回の実行結果（p値）は0.05を下回っているので、「別の標本でも平均体長が50cmになるものを得られることは滅多にない」、つまり「池の魚の平均体長は50cmと異なる」（有意差あり）と判断できます。

以上の計算はstats.ttest_1samp関数を用いると、一気にp値まで計算できます。引数にはデータと母平均を指定します

In [8]:
# stats.ttest_1samp()を使ってp値を計算
stats.ttest_1samp(data1, 50)

TtestResult(statistic=np.float64(3.2378191906168468), pvalue=np.float64(0.0030122666127776216), df=np.int64(29))

片側検定の実施
次に、「池の魚の平均体長は50cmより大きい」かどうかを調べてみましょう。こちらも有意差5％とします。「より大きい」とした場合は片側検定となります。仮説は以下のとおりです。

帰無仮説（H0）: 池の中の魚の平均体長は50cmである
対立仮説（H1）: 池の中の魚の平均体長は50cmより大きい
t値の計算までは両側検定と一緒です。片側検定では、図2のように分布の片側だけに棄却域を取りました。そのため、両側検定のときのように1から stats.t.cdf() を引いた計算結果を2倍する必要はありません。p値の計算は以下のようになります。



In [9]:
# p値を計算（片側検定：H1: 平均 > 50）
# t_value が観測された値。この値以上になる確率を求める (右側検定)
p_value = stats.t.sf(t_value, df=df)
p_value

np.float64(0.0015061333063888108)

こちらもp値が0.05を下回っているので 有意差あり とみなすことができます。よって、「池の魚の平均体長は50cmより大きい」と判断できます。

stats.ttest_1samp 関数は alternative 引数を指定することで片側検定も簡単に行えます。例えば、対立仮説が「より大きい」場合は alternative='greater'、「より小さい」場合は alternative='less' を指定します。

In [10]:
# stats.ttest_1samp()を使って片側検定のp値を計算 (H1: 平均 > 50)
stats.ttest_1samp(data1, 50, alternative='greater')

TtestResult(statistic=np.float64(3.2378191906168468), pvalue=np.float64(0.0015061333063888108), df=np.int64(29))

## 2変量データのt検定
『飼料A』で育った養殖魚と『飼料B』で育った養殖魚の体長』『「2歳」魚と「3歳」魚の体長』といった2変数の間で平均値に差があるか、否かについて考えてみましょう


In [11]:
# CSVファイルの読み込み
data2 = pd.read_csv('./hypothesis-testing-2.csv') #今回のCSVファイルには2つの列があるため、.squeeze("columns") は不要

In [12]:
# データの先頭5行を表示
data2.head()

,2-years-old,3-years-old
0,16.624345,14.600841
1,14.388244,14.984220
2,14.471828,14.606675
3,13.927031,14.401233
4,15.865408,14.627380


In [13]:
# 2変量の差を計算
data2['Difference'] = data2['3-years-old'] - data2['2-years-old']
data2.head()

,2-years-old,3-years-old,Difference
0,16.624345,14.600841,-2.023504
1,14.388244,14.984220,0.595977
2,14.471828,14.606675,0.134847
3,13.927031,14.401233,0.474201
4,15.865408,14.627380,-1.238028


差の平均値が0と異なれば、2歳魚と3歳魚で体長が異なると判断できるわけです。よって、対応のあるt検定では「差が0と異なるか」という1変数のt検定の実施が可能です。
帰無仮説と対立仮説について確認しましょう。対応のあるt検定では下記の仮説を立てることになりますが、これは括弧書きのように解釈できるという意味です。

帰無仮説（H0）: 2歳魚と3歳魚で体長は変わらない（2歳魚と3歳魚の体長の差は0）
対立仮説（H1）: 2歳魚と3歳魚で体長は異なる（2歳魚と3歳魚の体長の差は0ではない）
それではt検定を行いましょう。ここでは ttest_1samp() を使って一気にp値を求めることにします。引数には体長の差（Difference 列）と推定した母平均（0 cm）を指定します。

In [14]:
# 2歳魚と3歳魚の体長の差についてt検定を実施し、p値を計算
stats.ttest_1samp(data2['Difference'], 0)

TtestResult(statistic=np.float64(2.212896760339995), pvalue=np.float64(0.034931963004252754), df=np.int64(29))

p値が0.05を下回りましたので、有意差がある、つまり「2歳魚と3歳魚の体長は異なる」と判断できました。

## 対応のないt検定
対応のあるt検定では、それぞれの個体について2歳時点と3歳時点の2回計測を行った結果について解析しました。それでは、このデータが「2歳魚を集めたデータ」と「3歳魚を集めたデータ」ではあるものの 同じ個体の2歳時と3歳時の体長を測定したものではない とするとどうなるでしょうか。今度は、「平均値の差」について注目する必要があります。
対応のないt検定はstats.ttest_ind関数を用いると簡単に計算できます。
なお、引数の equal_var=False を指定すると、XとYの分散が等しくないと仮定したうえで検定が行われます。

In [15]:
# 2歳魚と3歳魚の体長について「対応のない」t検定を実施し、p値を計算
stats.ttest_ind(data2['2-years-old'], data2['3-years-old'], equal_var=False)

TtestResult(statistic=np.float64(-2.4038076492646794), pvalue=np.float64(0.019456469905707707), df=np.float64(57.74131958131711))

p値が0.05を下回ったので 有意差がある、つまり「2歳魚と3歳魚の体長は異なる」と判断できました。